<a href="https://colab.research.google.com/github/lingchm/datascience_notes/blob/master/dl_tensorflow/advanced_cv/L4_Simple_Object_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple Object Detection in Tensorflow

This lab will walk you through how to use object detection models available in [Tensorflow Hub](https://www.tensorflow.org/hub). In the following sections, you will:

* explore the Tensorflow Hub for object detection models
* load the models in your workspace
* preprocess an image for inference 
* run inference on the models and inspect the output

Let's get started!

In [1]:
# Install this package to use Colab's GPU for training
!apt install --allow-change-held-packages libcudnn8=8.4.1.50-1+cuda11.6

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following packages will be REMOVED:
  libcudnn8-dev
The following held packages will be changed:
  libcudnn8
The following packages will be upgraded:
  libcudnn8
1 upgraded, 0 newly installed, 1 to remove and 18 not upgraded.
Need to get 420 MB of archives.
After this operation, 3,369 MB disk space will be freed.
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  libcudnn8 8.4.1.50-1+cuda11.6 [420 MB]
Fetched 420 MB in 7s (61.3 MB/s)
(Reading database ... 159447 files and directories currently installed.)
Removing libcudnn8-dev (8.0.5.39-1+cuda11.1) ...
(Reading database ... 159425 files and directories currently installed.)
Preparing to unpack .../libcudnn8_8.4.1.50-1+cuda11.6_amd64.deb ...
Unpacking libcudnn8 (8.4.1.50-1+c

## Imports

In [3]:
import tensorflow as tf
import tensorflow_hub as hub
from PIL import Image
from PIL import ImageOps
import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO

### Download the model from Tensorflow Hub

Tensorflow Hub is a repository of trained machine learning models which you can reuse in your own projects. 
- You can see the domains covered [here](https://tfhub.dev/) and its subcategories. 
- For this lab, you will want to look at the [image object detection subcategory](https://tfhub.dev/s?module-type=image-object-detection). 
- You can select a model to see more information about it and copy the URL so you can download it to your workspace. 
- We selected a [inception resnet version 2](https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1)
- You can also modify this following cell to choose the other model that we selected, [ssd mobilenet version 2](https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2)

In [4]:
# you can switch the commented lines here to pick the other model

# inception resnet version 2
module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"

# You can choose ssd mobilenet version 2 instead and compare the results
#module_handle = "https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1"

#### Load the model

Next, you'll load the model specified by the `module_handle`.
- This will take a few minutes to load the model.

In [5]:
model = hub.load(module_handle)

#### Choose the default signature

Some models in the Tensorflow hub can be used for different tasks. So each model's documentation should show what *signature* to use when running the model. 
- If you want to see if a model has more than one signature then you can do something like `print(hub.load(module_handle).signatures.keys())`. In your case, the models you will be using only have the `default` signature so you don't have to worry about other types.

In [6]:
# take a look at the available signatures for this particular model
model.signatures.keys()

KeysView(_SignatureMap({'default': <ConcreteFunction pruned(images) at 0x7F5C7C842050>}))

Please choose the 'default' signature for your object detector.
- For object detection models, its 'default' signature will accept a batch of image tensors and output a dictionary describing the objects detected, which is what you'll want here.

In [7]:
detector = model.signatures['default']

### download_and_resize_image

This function downloads an image specified by a given "url", pre-processes it, and then saves it to disk.

In [8]:
def download_and_resize_image(url, new_width=256, new_height=256):
    '''
    Fetches an image online, resizes it and saves it locally.
    
    Args:
        url (string) -- link to the image
        new_width (int) -- size in pixels used for resizing the width of the image
        new_height (int) -- size in pixels used for resizing the length of the image
        
    Returns:
        (string) -- path to the saved image
    '''
    
    
    # create a temporary file ending with ".jpg"
    _, filename = tempfile.mkstemp(suffix=".jpg")
    
    # opens the given URL
    response = urlopen(url)
    
    # reads the image fetched from the URL
    image_data = response.read()
    
    # puts the image data in memory buffer
    image_data = BytesIO(image_data)
    
    # opens the image
    pil_image = Image.open(image_data)
    
    # resizes the image. will crop if aspect ratio is different.
    pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.ANTIALIAS)
    
    # converts to the RGB colorspace
    pil_image_rgb = pil_image.convert("RGB")
    
    # saves the image to the temporary file created earlier
    pil_image_rgb.save(filename, format="JPEG", quality=90)
    
    print("Image downloaded to %s." % filename)
    
    return filename

### Download and preprocess an image

Now, using `download_and_resize_image` you can get a sample image online and save it locally. 
- We've provided a URL for you, but feel free to choose another image to run through the object detector.
- You can use the original width and height of the image but feel free to modify it and see what results you get.

In [9]:
# You can choose a different URL that points to an image of your choice
image_url = "https://upload.wikimedia.org/wikipedia/commons/f/fb/20130807_dublin014.JPG"

# download the image and use the original height and width
downloaded_image_path = download_and_resize_image(image_url, 3872, 2592)

Image downloaded to /tmp/tmpec9a_wym.jpg.


### run_detector

This function will take in the object detection model `detector` and the path to a sample image, then use this model to detect objects and display its predicted class categories and detection boxes.
- run_detector uses `load_image` to convert the image into a tensor.

In [11]:
def load_img(path):
    '''
    Loads a JPEG image and converts it to a tensor.
    
    Args:
        path (string) -- path to a locally saved JPEG image
    
    Returns:
        (tensor) -- an image tensor
    '''
    
    # read the file
    img = tf.io.read_file(path)
    
    # convert to a tensor
    img = tf.image.decode_jpeg(img, channels=3)
    
    return img


def run_detector(detector, path):
    '''
    Runs inference on a local file using an object detection model.
    
    Args:
        detector (model) -- an object detection model loaded from TF Hub
        path (string) -- path to an image saved locally
    '''
    
    # load an image tensor from a local file path
    img = load_img(path)

    # add a batch dimension in front of the tensor
    converted_img  = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]
    
    # run inference using the model
    result = detector(converted_img)

    # save the results in a dictionary
    result = {key:value.numpy() for key,value in result.items()}

    # print results
    print("Found %d objects." % len(result["detection_scores"]))

    print(result["detection_scores"])
    print(result["detection_class_entities"])
    print(result["detection_boxes"])


### Run inference on the image

You can run your detector by calling the `run_detector` function. This will print the number of objects found followed by three lists: 

* The detection scores of each object found (i.e. how confident the model is), 
* The classes of each object found, 
* The bounding boxes of each object

You will see how to overlay this information on the original image in the next sections and in this week's assignment!

In [12]:
# runs the object detection model and prints information about the objects found
run_detector(detector, downloaded_image_path)

Found 100 objects.
[0.6531076  0.61050427 0.6016207  0.59267104 0.5922534  0.58160937
 0.5505509  0.49612293 0.47432286 0.47311208 0.44032288 0.4050859
 0.39809093 0.39374155 0.37155953 0.36171296 0.3615862  0.34719908
 0.33389986 0.3124711  0.28876108 0.25743204 0.25723046 0.25190163
 0.24772352 0.23448414 0.20432055 0.20342639 0.17997308 0.17972925
 0.17379068 0.16436897 0.16061224 0.1592036  0.15629333 0.15468292
 0.14733349 0.13605411 0.12753543 0.12559225 0.12122744 0.11804862
 0.11381513 0.11219216 0.11120925 0.09721024 0.09148445 0.08978613
 0.08883347 0.08618231 0.08337732 0.08093081 0.07997356 0.07750148
 0.07733559 0.07621965 0.07515509 0.07389456 0.07232054 0.07198709
 0.07109124 0.06931654 0.06837997 0.06433555 0.06249025 0.0623172
 0.06205211 0.05926957 0.05791891 0.05779758 0.05723752 0.05349667
 0.05305    0.05222971 0.04899192 0.04818314 0.0458142  0.04412691
 0.0434057  0.04290242 0.04265956 0.04166034 0.04086039 0.03967519
 0.039628   0.0394058  0.03872345 0.03773912 